In [56]:
# from google.colab import drive
# drive.mount('/content/drive')

In [57]:
data_path = r'C:\text-sentiment-classification-using-BERT\data\sentiment_analysis.csv'

In [58]:
from collections import defaultdict
import numpy as np

In [59]:
word2id = defaultdict(lambda: len(word2id))
PAD=word2id['']

In [60]:
sentences=[]
labels=[]
for line in open(data_path, encoding='utf-8'):
  line=line.strip()
  splits = line.split(',')
  id=splits[0]
  if id=='id':
    continue
  label=np.asarray([float(splits[1])])
  sent=splits[2]
  labels.append(label)
  sentences.append(sent)

data=[]

for i in range(len(sentences)):
  words=[]
  actual_words=[]
  for word in sentences[i].split(' '):

    actual_words.append(word)
    words.append(word2id[word])
  words=np.asarray(words)
  label=labels[i]
  data.append(((words,actual_words),label))


In [61]:
train=data[:4000]
val=data[4000:6000]
test=data[6000:]
print(len(train),len(val),len(test))

4000 2000 1920


In [62]:
from torch.utils.data import Dataset

In [63]:
def get_data(mode):
  if mode=='train':
    return train
  elif mode=='val':
    return val
  elif mode=='test':
    return test
  else:
    print('Mode is not set properly')
    assert(False)

In [64]:
class SADataset(Dataset):
  def __init__(self, mode):
    self.data=get_data(mode)
    self.len= len(self.data)
  def __getitem__(self, index):
    return self.data[index]
  def __len__(self):
    return self.len

In [65]:
import torch

In [66]:
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer

In [67]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [68]:
from torch.utils.data import DataLoader

In [69]:
def get_loader(mode, shuffle=True):
  dataset=SADataset(mode)

  def collate_fn(batch):

    batch=sorted(batch, key=lambda x: x[0][0].shape[0], reverse=True)
    labels = torch.cat([torch.from_numpy(sample[1]) for sample in batch], dim=0)
    sentences=pad_sequence([torch.LongTensor(sample[0][0]) for sample in batch], padding_value=PAD)

    SENT_LEN = sentences.size(0)

    labels = torch.cat([torch.from_numpy(sample[1]) for sample in batch], dim=0)

    bert_details=[]
    for sample in batch:
      text=" ".join(sample[0][1])

      encoded_bert_sent=bert_tokenizer.encode_plus(
        text, 
        max_length=SENT_LEN+2,
        add_special_tokens=True,
        truncation=True,
        pad_to_max_length=True)
      bert_details.append(encoded_bert_sent)

    bert_sentences = torch.LongTensor([sample["input_ids"] for sample in bert_details])
    bert_sentences_types = torch.LongTensor([sample["token_type_ids"] for sample in bert_details])
    bert_sentence_att_mask = torch.LongTensor([sample["attention_mask"] for sample in bert_details])

    return labels, bert_sentences, bert_sentences_types, bert_sentence_att_mask

  data_loader = DataLoader(dataset=dataset,
                           batch_size=64,
                           shuffle=shuffle,
                           collate_fn=collate_fn)

  return data_loader


In [70]:
train_data_loader = get_loader(mode='train', shuffle=True)

In [71]:
from transformers import BertConfig, BertModel

In [72]:
bertconfig = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
bertmodel = BertModel.from_pretrained('bert-base-uncased', config=bertconfig)

In [73]:
def get_bert_output(bertmodel, bert_sent, bert_sent_type, bert_sent_mask):
  bert_output=bertmodel(input_ids=bert_sent,
                                attention_mask=bert_sent_mask,
                                token_type_ids=bert_sent_type)

  bert_output=bert_output[0]

  batch_size = bert_output.shape[0]

  masked_output=torch.mul(bert_sent_mask.unsqueeze(2), bert_output)
  mask_len=torch.sum(bert_sent_mask, dim=1, keepdim=True)
  bert_output=torch.sum(masked_output, dim=1, keepdim=False) / mask_len

  return bert_output

In [74]:
import torch.nn as nn

In [75]:
class LSTM(nn.Module):
  def __init__(self):
    super(LSTM, self).__init__()
    rnn = nn.LSTM
    hidden_size=16
    self.text_rnn1=rnn(768, hidden_size, bidirectional=False)
    self.text_rnn2=rnn(hidden_size, hidden_size, bidirectional=False)

    self.fc = nn.Linear(hidden_size, 2)

  def forward(self, x):
    _, (vec1, _) = self.text_rnn1(x)
    _, (vec2, _) = self.text_rnn2(vec1)
    vec2=torch.squeeze(vec2, dim=0)
    print('vec2:', vec2.shape)
    final_out = self.fc(vec2)
    print('f:', final_out.shape)

    return final_out

In [76]:
my_model=LSTM()

In [77]:
from torch import optim

In [78]:
loss_fn = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(filter(lambda p: p.requires_grad, my_model.parameters()), lr=0.001)

In [79]:
for batch in train_data_loader:
  label, bert_sent, bert_sent_type, bert_sent_mask = batch

  bert_output = get_bert_output(bertmodel, bert_sent, bert_sent_type, bert_sent_mask)

  bert_output = torch.unsqueeze(bert_output,dim=0)


  pred = my_model(bert_output)

  print(pred[:5], label[:5])
  assert (pred.size(1) == 2), "Output size mismatch. Expected size (batch_size, 2)."

  # assert(False)

vec2: torch.Size([64, 16])
f: torch.Size([64, 2])
tensor([[-0.0141, -0.2251],
        [-0.0272, -0.2420],
        [-0.0345, -0.2382],
        [-0.0205, -0.2538],
        [-0.0041, -0.2297]], grad_fn=<SliceBackward0>) tensor([1., 0., 1., 0., 1.], dtype=torch.float64)
vec2: torch.Size([64, 16])
f: torch.Size([64, 2])
tensor([[-0.0154, -0.2171],
        [-0.0250, -0.2449],
        [-0.0219, -0.2428],
        [-0.0258, -0.2444],
        [-0.0146, -0.2448]], grad_fn=<SliceBackward0>) tensor([0., 0., 0., 0., 0.], dtype=torch.float64)
vec2: torch.Size([64, 16])
f: torch.Size([64, 2])
tensor([[-0.0324, -0.2557],
        [-0.0185, -0.2344],
        [-0.0242, -0.2281],
        [-0.0085, -0.2199],
        [-0.0191, -0.2337]], grad_fn=<SliceBackward0>) tensor([0., 0., 0., 1., 1.], dtype=torch.float64)
vec2: torch.Size([64, 16])
f: torch.Size([64, 2])
tensor([[-0.0201, -0.2369],
        [-0.0191, -0.2201],
        [-0.0273, -0.2370],
        [-0.0195, -0.2478],
        [-0.0114, -0.2409]], grad_fn=